In [1]:
import time
from selenium import webdriver
import pandas as pd
from selenium.webdriver.chrome.service import Service # not used in this project because it doesn't work for some reason
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import os
import subprocess

In [2]:
address = ["https://dubaioutsourcecity.ae/the-community/community-directory", 
"https://dic.ae/the-community/community-directory"]

In [3]:
# note: you have to install geckodriver using "conda install -c conda-forge geckodriver" because selenium firefox 
# relies on
# geckodriver to work

## Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")


In [4]:
# Set path to chromedriver as per your configuration
# webdriver_service = Service("/home/michael/chromedriver/stable/chromedriver") # for unknown reason doesn't work

chrome_driver_path = "/home/michael/chromedriver/stable/chromedriver"

# kill all open chromedriver instance before opening a new one to conserve system resource
k_ = subprocess.run("ps aux | awk '/chrome/ { print $2 } ' | xargs kill -9", shell=True)
print(k_)


# Choose Chrome Browser
browser = webdriver.Chrome(chrome_driver_path, options=chrome_options)

# browser.implicitly_wait(20)   # this allows the webdriver instance to wait for a period of time after loading a page
                            # before searching for an element

your 131072x1 screen size is bogus. expect trouble
kill: (30405): No such process


CompletedProcess(args="ps aux | awk '/chrome/ { print $2 } ' | xargs kill -9", returncode=-9)


In [5]:
# get web page
# Create lists to hold the different information about the listed companies from the website directory

category1 = []
name1 = []
description1 = []
phone1 = []
website1 = []
location1 = []

for add_ in address:

    category = []
    name = []
    description = []
    phone = []
    website = []
    location = []

    browser.get(add_)

    button_presence = True
    while button_presence:
        try:
            # wait for the load more button to be shown in the web page
            button= WebDriverWait(browser,30).until(EC.visibility_of_element_located((By.CSS_SELECTOR,
            "button.cta-primary[title='Load More']")))

            if button.is_displayed():   # checks if 'Load More' button is displayed
                browser.execute_script(
                    "arguments[0].click();", WebDriverWait(browser, 20).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.cta-primary[title='Load More']"))))
                
                # browser.execute_script("arguments[0].click();", *args) this uses javascript to click button
        
        # catch TimeoutException error when the 'Load More' button is no longer displayed on the webpage and 
        # quit the loop

        except TimeoutException:
            button_presence = False

    companies_breakdown = browser.find_elements_by_css_selector("div.box-content")

    for i in companies_breakdown:
        try:
            cat_ = i.find_element_by_css_selector("p.item-category").text
            category.append(cat_)
        except NoSuchElementException:
            category.append("")

        try:
            name_ = i.find_element_by_css_selector("p.item-name").text
            name.append(name_)
        except NoSuchElementException:
            name.append("")

        try:
            desc_ = i.find_element_by_css_selector("p.item-desc").text
            description.append(desc_)
        except NoSuchElementException:
            description.append("")

        # check if the company's phone contact is listed
        try:
            mobile = i.find_element_by_css_selector("a.cta-primary.item-phone").get_attribute("href")
            phone.append(mobile)
        except NoSuchElementException:
            phone.append("")
        
        # check if the company's website is listed
        try:
            web = i.find_element_by_css_selector("a.cta-primary.--has-icon").get_attribute("href")
            website.append(web)
        except NoSuchElementException:
            website.append("")

        loc_ = add_.replace("https://","")
        loc_ = loc_.split("/")[0]
        location.append(loc_)
    
    category1.extend(category)
    name1.extend(name)
    description1.extend(description)
    phone1.extend(phone)
    website1.extend(website)
    location1.extend(location)


# button = browser.find_element_by_css_selector("button.cta-primary[title='Load More']").click()



# buttons = WebDriverWait(browser,10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,
# "button.cta-primary[title='Load More']")))

In [8]:
# create a pandas dataFrame from the lists

companies_df = pd.DataFrame(list(zip(location1,name1,category1,description1,phone1, website1)), 
columns =['location','name','category','description','phone','website'])

In [9]:
# export dataframe to csv files
companies_df.to_csv('companies_list1.csv')